1. Здійсніть імпорт необхідних пакетів.

In [7]:
!pip install category_encoders
!pip install gdown
import gdown
import warnings
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
import category_encoders as ce
from scipy.stats import skew, kurtosis

2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv набори даних, доступні в репозиторії. Цільова змінна — рівень заробітної плати співробітників (Salary).

In [8]:
url = 'https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ'
output = 'mod_04_hw_train_data.csv'
gdown.download(url, output, quiet=False)
data_train = pd.read_csv(output)

url_val = 'https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a'
output_val = 'mod_04_hw_valid_data.csv'
gdown.download(url_val, output_val, quiet=False)
data_val = pd.read_csv(output_val)

Downloading...
From: https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ
To: /Users/julia/Developer/GitDownload/Gits_DataScienceMasters/Machine_Learning_Masters/mod_04_hw_train_data.csv
100%|██████████| 17.2k/17.2k [00:00<00:00, 11.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a
To: /Users/julia/Developer/GitDownload/Gits_DataScienceMasters/Machine_Learning_Masters/mod_04_hw_valid_data.csv
100%|██████████| 549/549 [00:00<00:00, 878kB/s]


In [9]:
print(data_train.head())
print(data_val.head(7))

                 Name  Phone_Number  Experience Qualification University  \
0  Jennifer Hernandez  120-602-1220         3.0           Msc      Tier2   
1      Timothy Walker  840-675-8650         5.0           PhD      Tier2   
2         David Duran  556-293-8643         5.0           Msc      Tier2   
3       Gloria Ortega  463-559-7474         3.0           Bsc      Tier3   
4      Matthew Steele  968-091-7683         5.0           Bsc      Tier2   

     Role Cert Date_Of_Birth  Salary  
0     Mid  Yes    25/08/1972   98000  
1  Senior  Yes    03/12/2013  135500  
2  Senior  Yes    19/07/2002  123500  
3     Mid   No    19/02/1970   85000  
4  Senior  Yes    20/02/1970  111500  
              Name  Phone_Number  Experience Qualification University    Role  \
0   Alvaro Johnson  320-636-8883           7           Bsc      Tier1  Senior   
1    Austin Powers  903-121-1691           2           Msc      Tier1     Mid   
2      Joshua Phil  673-972-2453           3           Bsc      Ti

3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

In [10]:
numerical_features = ['Experience']
categorical_features = ['Qualification', 'Role', 'Cert']

In [11]:
columns_to_drop = ['Date_Of_Birth', 'University', 'Phone_Number', 'Name']

df_train = data_train.drop(columns=columns_to_drop)
df_val = data_val.drop(columns=columns_to_drop)

print("Remaining columns after dropping:")
print(df_train.columns)

print("Remaining  df_val columns after dropping:")
print(df_val.columns)

Remaining columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')
Remaining  df_val columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')


In [12]:
# Заповнення числових стовпців середнім значенням
for column in numerical_features:
    df_train[column] = df_train[column].fillna(df_train[column].mode()[0])

# Заповнення категоріальних стовпців модою
for column in categorical_features:
    df_train[column] = df_train[column].fillna(df_train[column].mode()[0])

# Перевірка після заповнення
print(df_train.isnull().sum())
print(df_train.head(3))

Experience       0
Qualification    0
Role             0
Cert             0
Salary           0
dtype: int64
   Experience Qualification    Role Cert  Salary
0         3.0           Msc     Mid  Yes   98000
1         5.0           PhD  Senior  Yes  135500
2         5.0           Msc  Senior  Yes  123500


In [13]:
Q1 = df_train['Experience'].quantile(0.25)
Q3 = df_train['Experience'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

train_data = df_train[(df_train['Experience'] >= lower_bound) & (df_train['Experience'] <= upper_bound)]

print("Розмірність після очищення:", df_train.shape)
print(df_train.head(3))

Розмірність після очищення: (249, 5)
   Experience Qualification    Role Cert  Salary
0         3.0           Msc     Mid  Yes   98000
1         5.0           PhD  Senior  Yes  135500
2         5.0           Msc  Senior  Yes  123500


4. Виконайте обробку числових ознак (трансформацію / нормалізацію за допомогою об’єктів StandardScaler або PowerTransformer з пакета sklearn) і кодування категоріальних (наприклад, за допомогою об’єктів OneHotEncoder / TargetEncoder з пакета sklearn або об’єктів з пакета category_encoders).

In [19]:
# Define feature lists
numerical_features = ['Experience']
categorical_features = ['Qualification', 'Role', 'Cert']

# Separate features and target variable
X_train = df_train.drop('Salary', axis=1)
y_train = df_train['Salary']

# Use numerical features as they are
X_train_numerical = X_train[numerical_features].values

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical = encoder.fit_transform(X_train[categorical_features]).toarray()

# Combine numerical and categorical features
X_train_transformed = np.hstack((X_train_numerical, X_train_categorical))

# Get feature names for the categorical features
categorical_feature_names = encoder.get_feature_names_out(categorical_features)

# Combine all feature names
all_feature_names = numerical_features + list(categorical_feature_names)

# Create a DataFrame for exploration
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=all_feature_names)

   Experience  Qualification_Bsc  Qualification_Msc  Qualification_PhD  \
0         3.0                0.0                1.0                0.0   
1         5.0                0.0                0.0                1.0   
2         5.0                0.0                1.0                0.0   
3         3.0                1.0                0.0                0.0   
4         5.0                1.0                0.0                0.0   

   Role_Junior  Role_Mid  Role_Senior  Cert_No  Cert_Yes  
0          0.0       1.0          0.0      0.0       1.0  
1          0.0       0.0          1.0      0.0       1.0  
2          0.0       0.0          1.0      0.0       1.0  
3          0.0       1.0          0.0      1.0       0.0  
4          0.0       0.0          1.0      0.0       1.0  


In [21]:
# Now you can explore X_train_transformed_df
print(X_train_transformed_df.head(20))

    Experience  Qualification_Bsc  Qualification_Msc  Qualification_PhD  \
0          3.0                0.0                1.0                0.0   
1          5.0                0.0                0.0                1.0   
2          5.0                0.0                1.0                0.0   
3          3.0                1.0                0.0                0.0   
4          5.0                1.0                0.0                0.0   
5          5.0                0.0                0.0                1.0   
6          5.0                0.0                0.0                1.0   
7          5.0                1.0                0.0                0.0   
8          3.0                1.0                0.0                0.0   
9          5.0                0.0                1.0                0.0   
10         2.0                0.0                0.0                1.0   
11         5.0                1.0                0.0                0.0   
12         4.0           

5. Побудуйте модель за допомогою об'єкта KNeighborsRegressor з пакета sklearn.

In [22]:
model = KNeighborsRegressor(n_neighbors=4, weights='uniform', metric='manhattan')  # Вибір кількості сусідів
model.fit(X_train_transformed, y_train)

y_pred_train = model.predict(X_train_transformed)
mape = mean_absolute_percentage_error(y_train, y_pred_train)

print(f'Обчислена MAPE: {mape:.2%}')

Обчислена MAPE: 3.73%


6. Виконайте аналогічні етапи обробки і підготовки даних для валідаційного набору, подібно до того, як ми це робили у темі «Метод опорних векторів (SVM)» в розділі «Практика застосування SVM-класифікатора. Навчання й оцінка моделі. Приклад використання моделі)» .

In [23]:
X_valid = df_val.drop('Salary', axis=1)
y_valid = df_val['Salary']

X_valid_numerical = scaler.transform(X_valid[numerical_features])

X_valid_categorical = encoder.transform(X_valid[categorical_features]).toarray()

X_valid_transformed = np.hstack((X_valid_numerical, X_valid_categorical))

y_pred_valid = model.predict(X_valid_transformed)

mape_valid = mean_absolute_percentage_error(y_valid, y_pred_valid)
print(f'MAPE для валідаційного набору: {mape_valid:.2%}')

NameError: name 'scaler' is not defined

7. Отримайте прогноз заробітної плати для нових об’єктів (працівників) із валідаційного набору, розрахуйте доцільні метрики точності регресійної моделі.